<a href="https://colab.research.google.com/github/NatalyGri/DSL/blob/main/Job_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Используя представление грамматики, предложенное в Job #2 написать:
1.   Программу, генерирующую функцию FIRST
2.   Программу, генерирующую функцию FOLLOW

Грамматика:
```
{'toks': set(token), 'vars': dict(var: definition), 'hvar': var}
token : (class, value)
class : int
value : str
var : str                 # имя нетерминала
definition : list(rule)
rule : list(var | token)  # правая часть правила
```



**FIRST(A)** - все терминалы, с которых могут начинаться всевозможные выводы из A.
Чтобы найти функцию FIRST для нашей грамматики, мы просматриваем каждое правило для нетерминала следующим образом:
1.   Если правило начинается с терминала, мы добавляем этот терминал в множество "first" и переходим к следующему правилу.
2.   Если правило начинается с нетерминала, мы вызываем функцию FIRST для этого нетерминала и анализируем полученный набор. Если полученное множество содержит пустой символ, переходим к следующему символу правила, а полученное множество добавляем к "first". Если в наборе нет пустого символа, то переходим к следующему правилу.

В конце концов, мы применим функцию FIRST для каждого нетерминала и найдем результирующий набор для каждого нетерминала.

In [7]:
def firsts_set(grammar, empt): #функция возвращает множество "first" для каждого нетерминала
  nterminals_firsts = dict() 
  nterminals = grammar['vars']

  def first(nterm): #функция возвращает множество "first" для конкретного нетерминала
    terminals_set = set() 
    for rule in nterminals[nterm]:
      for symbol in rule:
        if symbol not in nterminals:
          terminals_set.add(symbol)
          break;
        else:
          if symbol not in nterminals_firsts.keys():
            nterminals_firsts[symbol] = first(symbol)
          terminals_set.update(nterminals_firsts[symbol])
          if empt not in terminals_set:
            break;
    return terminals_set

  for nterm in nterminals.keys():
    if nterm not in nterminals_firsts:
      nterminals_firsts[nterm] = set(first(nterm))
  return nterminals_firsts

**FOLLOW(A)** - все терминальные символы, которые встречаются справа от нетерминала A во всех возможных сентенциальных формах.

Рассмотрим все продуцинные правила типа A → αBβ:

1.   Находим набор FIRST(β) и добавляем его к множеству "follow" для нетерминального B.
2.   Если FIRST(β) содержит пустой символ, мы также добавляем множество "follow" нетерминала A к множеству "follow" нетерминала B (FOLLOW(B) += FOLLOW(A)).
3.   Далее, мы переходим к следующему правилу и повторяем наши действия, пока множество "follow" нетерминала B увеличивается.

In [8]:
def follows_set(grammar, empt):
  follows = dict() 
  firsts = firsts_set(grammar, empt)
  terminals = grammar['toks']
  nterminals = grammar['vars']
  nonterminal = grammar['hvar']

  for nterm in nterminals:
    follows[nterm] = set()

  #Вспомогательна функция для нахождения множества крайних левых токенов, выводимых из данной последовательности

  def firsts_for_sequence(sequence):
    tokens = set()
    for symbol in sequence:
      if symbol in terminals: 
        tokens.add(symbol)
        return tokens
      else:
        tokens.update(firsts[symbol])
        if empt not in firsts[symbol]:
          return tokens
    return tokens


  follows[nonterminal].add('$') # '$' - как конец строки
  find_new_follow = True
  while find_new_follow:
    find_new_follow = False
    for nterm, definition in nterminals.items():
      for rule in definition:
        while len(rule):
          left = rule.pop(0) 
          if left not in terminals:
            befor = len(follows[left]) 
            f_set = firsts_for_sequence(rule) 
            follows[left].update(f_set.difference(empt)) 
            if empt in f_set or len(rule) == 0: 
              follows[left].update(follows[nterm]) 
            after = len(follows[left])
            if not (befor == after):
              find_new_follow = True
  return follows

In [9]:
grammar = {
    'toks': set( [
        (0, 'e'), 
        (1, 'v'), 
        (2, 'n'), 
        (3, '+'),
        (4, '*'),
        (5, '('),
        (6, ')')
    ] ),

    'vars': {
        'S' : [['T', 'C']],

        'C' : [[(3, '+'), 'T', 'C'],
               [(0, 'e')]],
             
        'T' : [['F', 'D'],
               [(0, 'e')]],
             
        'D' : [[(4, '*'), 'F', 'D'],
               [(0, 'e')]],
             
        'F' : [[(1, 'v')],
               [(2, 'n')],
               [(5, '('), 'S', (6, ')')]],
    },
    'hvar': 'S'
}

print("Result FIRST: ", firsts_set(grammar, (0, 'e')))
print("Result FOLLOW: ", follows_set(grammar, (0, 'e')))

Result FIRST:  {'F': {(1, 'v'), (2, 'n'), (5, '(')}, 'T': {(1, 'v'), (2, 'n'), (5, '('), (0, 'e')}, 'C': {(0, 'e'), (3, '+')}, 'S': {(1, 'v'), (2, 'n'), (5, '('), (0, 'e'), (3, '+')}, 'D': {(4, '*'), (0, 'e')}}
Result FOLLOW:  {'S': {'$', (6, ')')}, 'C': {'$'}, 'T': {'$', (0, 'e'), (3, '+')}, 'D': {'$', (0, 'e'), (3, '+')}, 'F': {'$', (0, 'e'), (4, '*'), (3, '+')}}
